In [1]:
# the plan is simple
# mew and looksmaxx
# take into account all factors of a pokemon (except name, weight, height)
# then predict the type
# THEN
# i have no idea what model to use BUT
# we test accuracy using newer gen pokemon
import pandas as pd
from sklearn import preprocessing
import numpy as np
raw_data = pd.read_csv('pokemon_data.csv')
print(raw_data.head(0))

Empty DataFrame
Columns: [dexnum, name, generation, type1, type2, species, height, weight, ability1, ability2, hidden_ability, hp, attack, defense, sp_atk, sp_def, speed, total, ev_yield, catch_rate, base_friendship, base_exp, growth_rate, egg_group1, egg_group2, percent_male, percent_female, egg_cycles, special_group]
Index: []

[0 rows x 29 columns]


In [2]:
#so what we do here is get RID of all the cringe stuff that has no effect (i think) on typings WE DO NOT TALK ABOUT ABILITYS NUH UH I AINT MAKING A DICTIONARY
overwrite_data = raw_data
overwrite_data= overwrite_data[["type1", "type2", #"ability1", "ability2",
                                 "hp", "attack", "defense", "sp_atk", "sp_def", "speed"]]

raw_data = overwrite_data
overwrite_data = overwrite_data[overwrite_data['type2'].isna()] # here we get rid off all pokemon that have 2 types, 
                                                                #bc we only doing pokemon with 1 type
overwrite_data= overwrite_data[["type1", #"ability1", "ability2",
                                 "hp", "attack", "defense", "sp_atk", "sp_def", "speed"]] # remove the type2 collum
raw_data = overwrite_data

print(raw_data.head(10))


     type1  hp  attack  defense  sp_atk  sp_def  speed
3     Fire  39      52       43      60      50     65
4     Fire  58      64       58      80      65     80
6    Water  44      48       65      50      64     43
7    Water  59      63       80      65      80     58
8    Water  79      83      100      85     105     78
9      Bug  45      30       35      20      20     45
10     Bug  50      20       55      25      25     30
18  Normal  30      56       35      25      35     72
19  Normal  55      81       60      50      70     97
22  Poison  35      60       44      40      54     55


In [3]:
#NOW what we do is define X and Y, Y is what we are predicting
Y = raw_data[["type1"]]
Y = Y.pop("type1") # get RID of the title 

# okie now we turn types into numbers
# set up thingy for later zzz
typenumbers = {0:"Normal", 1:"Fighting", 2:"Flying", 3:"Poison", 4:"Ground", 5:"Rock", 6:"Bug", 7:"Ghost", 8:"Steel", 9:"Fire", 10:"Water", 11:"Grass", 
               12:"Electric", 13:"Psychic", 14:"Ice", 15:"Dragon", 16:"Dark", 17:"Fairy"} # for converting typings to numbers
good_typenumbers = {v: k for k, v in typenumbers.items()} # i REALLY dont want to rewrite the previous line

raw_data = raw_data.replace(to_replace=good_typenumbers)
Y = Y.replace(to_replace=good_typenumbers) # swap out the string of types with the number types

print(raw_data.head(10))
print(Y)
raw_data = raw_data[[#"type1", #"ability1", "ability2",
                                 "hp", "attack", "defense", "sp_atk", "sp_def", "speed"]] # finally, we have our X
from sklearn import preprocessing
#raw_data = preprocessing.StandardScaler().fit(raw_data).transform(raw_data) for some reason leaving this out makes the model more accuate?

    type1  hp  attack  defense  sp_atk  sp_def  speed
3       9  39      52       43      60      50     65
4       9  58      64       58      80      65     80
6      10  44      48       65      50      64     43
7      10  59      63       80      65      80     58
8      10  79      83      100      85     105     78
9       6  45      30       35      20      20     45
10      6  50      20       55      25      25     30
18      0  30      56       35      25      35     72
19      0  55      81       60      50      70     97
22      3  35      60       44      40      54     55
3        9
4        9
6       10
7       10
8       10
        ..
976     10
981      0
998      7
1016    11
1023     0
Name: type1, Length: 499, dtype: int64


/var/folders/dh/vvmqj_jj3f77xdfpf21vgmgh0000gp/T/ipykernel_4535/204346955.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data = raw_data.replace(to_replace=good_typenumbers)
/var/folders/dh/vvmqj_jj3f77xdfpf21vgmgh0000gp/T/ipykernel_4535/204346955.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = Y.replace(to_replace=good_typenumbers) # swap out the string of types with the number types


In [4]:
#Test set YIPEEE
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(raw_data, Y, train_size = 0.7, random_state=12345)
print('X_train.shape=', trainX.shape, 'Y_train.shape=', trainY.shape)
print(testX)


X_train.shape= (349, 6) Y_train.shape= (349,)
     hp  attack  defense  sp_atk  sp_def  speed
509  64      88       50      88      50    106
284  60      40       60      40      60     35
352  44      75       35      63      33     45
581  36      50       50      65      60     44
100  60      50       70      80      80    150
..   ..     ...      ...     ...     ...    ...
820  38      47       35      33      35     57
32   61      72       57      55      55     65
830  42      40       55      40      45     48
561  38      30       85      55      65     30
534  50      50       40      50      40     64

[150 rows x 6 columns]


In [5]:
# oh boy oh boy probabilities i do love probabilities asdljfhas;ldjfhasdlfjhasdfl;kjahsdflkjahsdflkjahsdflkajhsdf
# getting ready for softmaxxing
# NVM this is easy if sklearn works

# Update: this is so cringe, sklearn is not doing what i want it to do :(
# I sorta wanna do it by myself but idk if i have time
# to understand: Sklearn only likes numbers which is kinda cringe, which it really dosent need to
# Like lets say the ability is blaze, it should see that the main type is fire type, so when checking new data, it is more likely to be a fire type
# sklearn says nuh uh its gotta be numbers but that just dosent work
# also why do the predicted classes have to be numbers
# so cringe smh
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
#scale = preprocessing.StandardScaler().fit(raw_data)
#scaled_data = scale.transform(raw_data)
#lr = LogisticRegression(random_state=0, max_iter=1000).fit(scaled_data, Y)
#probability=lr.predict_proba(raw_data)

In [6]:
#warnings are for YAPPERS

peak_model = SVC(kernel = "linear", gamma = 0.5, probability = True)
peak_model.fit(raw_data, Y) 
yhat = peak_model.predict(testX)


#that was a lot eaisier than i thought i would be



In [7]:
#based
print(classification_report(testY, yhat))
print("Coeficients: %s" % (peak_model.coef_))

              precision    recall  f1-score   support

           0       0.37      0.68      0.48        19
           1       0.41      0.58      0.48        12
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.50      1.00      0.67         2
           5       1.00      0.60      0.75         5
           6       0.75      0.38      0.50         8
           7       1.00      0.40      0.57         5
           9       0.20      0.10      0.13        10
          10       0.24      0.62      0.35        24
          11       0.00      0.00      0.00        16
          12       0.33      0.07      0.12        14
          13       0.17      0.29      0.21         7
          14       0.00      0.00      0.00         5
          15       1.00      0.50      0.67         4
          16       0.00      0.00      0.00         3
          17       1.00      0.10      0.18        10

    accuracy              

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [8]:
#edgemaxxing?
print("This is the swag pokemon generator")
print("its pretty hip and cool yk")
print("this is only based on stats bc sklearn is kinda stupid")
hp = input("hp: ")
atk = input("attack: ")
defe = input("defense: ")
spatk = input("special attack: ")
spdefe = input("special defence: ")
speed = input("speed: ")



This is the swag pokemon generator
its pretty hip and cool yk
this is only based on stats bc sklearn is kinda stupid


In [9]:
newpokemonlist = np.array([hp, atk, defe, spatk, spdefe, speed]) # make a new data row(?) with our input variables
newpokemonlist = newpokemonlist.reshape(1, -1) # make the thing happy
#newpokemonlist = preprocessing.StandardScaler().fit(newpokemonlist).transform(newpokemonlist)


predictedtype = peak_model.predict(newpokemonlist)
print("A pokemon with %s hp, %s attack, %s defence, %s special attack, %s special defence, and %s speed has a predicted type of:" % (hp, atk, defe, spatk, spdefe, speed))
print(typenumbers[predictedtype[0]])

A pokemon with 500 hp, 10 attack, 10 defence, 10 special attack, 10 special defence, and 10 speed has a predicted type of:
Normal


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
